<a href="https://colab.research.google.com/github/GitSantiagopf/GenreRecognition/blob/main/GenreRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [1]:
#Descargar la base de datos
!gdown --id "1f-qUobh_tWoEIhB35uSi3cECSF45yFcl"
!wget https://drive.google.com/file/d/1f-qUobh_tWoEIhB35uSi3cECSF45yFcl

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1f-qUobh_tWoEIhB35uSi3cECSF45yFcl
To: /content/canciones.rar
100% 560M/560M [00:07<00:00, 75.0MB/s]
--2023-04-20 00:48:29--  https://drive.google.com/file/d/1f-qUobh_tWoEIhB35uSi3cECSF45yFcl
Resolving drive.google.com (drive.google.com)... 108.177.126.139, 108.177.126.138, 108.177.126.102, ...
Connecting to drive.google.com (drive.google.com)|108.177.126.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/1f-qUobh_tWoEIhB35uSi3cECSF45yFcl/ [following]
--2023-04-20 00:48:29--  https://drive.google.com/file/d/1f-qUobh_tWoEIhB35uSi3cECSF45yFcl/
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Tempora

In [5]:
#Canción de prueba
!gdown --id "1I_THlsDbidYfHm3H2UmunHDBENT615me"
!wget https://drive.google.com/file/d/1I_THlsDbidYfHm3H2UmunHDBENT615me

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1I_THlsDbidYfHm3H2UmunHDBENT615me
To: /content/Rels.mp3
100% 2.43M/2.43M [00:00<00:00, 138MB/s]
--2023-04-20 01:01:58--  https://drive.google.com/file/d/1I_THlsDbidYfHm3H2UmunHDBENT615me
Resolving drive.google.com (drive.google.com)... 108.177.126.102, 108.177.126.139, 108.177.126.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.126.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/1I_THlsDbidYfHm3H2UmunHDBENT615me/ [following]
--2023-04-20 01:01:58--  https://drive.google.com/file/d/1I_THlsDbidYfHm3H2UmunHDBENT615me/
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily

In [2]:
!unrar x canciones.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from canciones.rar

Creating    canciones                                                 OK
Creating    canciones/Clasica                                         OK
Extracting  canciones/Clasica/André Rieu - Ballade pour Adeline.mp3        0%  OK 
Extracting  canciones/Clasica/Antonio Vivaldi La Stravaganza.mp3           1%  2%  3%  OK 
Extracting  canciones/Clasica/Beethoven – Pathetique Sonata 2nd Movement.mp3       4%  OK 
Extracting  canciones/Clasica/Beethoven – Sonata No.17 Tempest 3rd Movement.mp3       5%  6%  OK 
Extracting  canciones/Clasica/Brahms - Hungarian Dances No. 5 _ 6 conducted by Maciej Tomasiewicz.mp3       7%  8%  OK 
Extracting  canciones/Clasica/Chopin  Nocturne op9 No2.mp3                 9%  OK 
Extracting  canciones/Clasica/Chopin  Spring Waltz Mariage dAmour Please Read Description.mp3       9% 

In [6]:
import os
import soundfile as sf
import numpy as np
import random

segment_size = 5 * 48000  # 5 segundos de audio
segments_per_song = 4  # 4 segmentos por canción

#Generos de música a clasificar
genres = ['Clasica', 'Pop', 'Metal']


#Se lee cada archivo de audio de cada género y se divide en los segmentos establecidos.
#Luego se aplica la Transformada de Fourier a cada segmento 
#y se almacena el espectro resultante en un diccionario donde cada género tiene una lista de espectros de sus canciones.
song_spectra = {}
for genre in genres:
    song_spectra[genre] = []
    genre_dir = os.path.join('canciones', genre)
    for song_path in os.listdir(genre_dir):
        song, sample_rate = sf.read(os.path.join(genre_dir, song_path))
        song_length = len(song)
        for i in range(segments_per_song):
            segment_start = random.randint(0, song_length - segment_size)
            segment_end = segment_start + segment_size
            if segment_end > song_length:
                segment = np.zeros(segment_size)
                segment[:song_length - segment_start] = song[segment_start:]
            else:
                segment = song[segment_start:segment_end]
            spectrum = np.abs(np.fft.fft(segment))
            song_spectra[genre].append(spectrum)


#Se construye una base de datos de espectros de Fourier para cada género musical a partir de los segmentos de audio obtenidos anteriormente.
#Para cada género, se calcula la media de todos los espectros de Fourier obtenidos en los segmentos de audio pertenecientes a ese género. 
#El resultado es un diccionario donde la clave es el género y el valor es el espectro promedio correspondiente a ese género.
spectra_db = {}
for genre in genres:
    spectra_db[genre] = np.mean(song_spectra[genre], axis=0)



# se define una función que clasifica una nueva canción en uno de los tres géneros definidos anteriormente.
# La función toma como entrada el path de la canción a clasificar y devuelve el género más probable de esa canción.
# se lee el archivo de audio y se obtiene su longitud. Luego, se dividen los segmentos de la misma manera que en la sección anterior.
#Para cada segmento, se calcula su espectro de Fourier y se almacena en una lista llamada song_spectra. 
#Luego, se calcula la distancia promedio entre los espectros de la canción y los espectros promedio de cada género en la base de datos.
#El resultado es un diccionario donde la clave es el género y el valor es la distancia promedio entre los espectros de la canción y los espectros promedio del género correspondiente.
#Finalmente, se devuelve el género con la distancia promedio más pequeña, lo que indica que es el género más probable de la canción.

def classify_song(song_path):
    song, sample_rate = sf.read(song_path)
    song_length = len(song)
    segment_start_indices = random.sample(range(0, song_length - segment_size + 1, segment_size), segments_per_song)
    song_segments = []
    for start_idx in segment_start_indices:
        segment_end = start_idx + segment_size
        if segment_end > song_length:
            segment = np.zeros(segment_size)
            segment[:song_length - start_idx] = song[start_idx:]
        else:
            segment = song[start_idx:segment_end]
        song_segments.append(segment)
    song_spectra = []
    for segment in song_segments:
        spectrum = np.abs(np.fft.fft(segment))
        song_spectra.append(spectrum)
    distances = {}
    for genre in genres:
        distances[genre] = np.mean([np.linalg.norm(spectrum - spectra_db[genre]) for spectrum in song_spectra])
    return min(distances, key=distances.get)

In [ ]:
genre = classify_song('Rels.mp3')
print(genre)

Pop
